# Very simple baseline
Use the most populat items as a baseline prediction. This is a very, very simple baseline to check our future results

In [7]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from metrics import apk, mapk, precision_at_k, recall_at_k
from utils import get_train_test_data_split, pd_to_scr_matrix

In [8]:
data = pd.read_csv('../../data/recsys_data.csv')
data['date'] =pd.to_datetime(data['date'])

In [10]:
train, test  = get_train_test_data_split(data, val_start = pd.Timestamp('2020-06-27 00:00:00'), test_days= 1 )

In [12]:
train.head()

,user,survey,status,date
0,46120029,100000000,0,2020-06-25 16:08:06
1,61685181,100000001,0,2020-06-24 12:29:43
2,61900560,100000002,0,2020-06-24 20:35:37
3,26620554,100000003,0,2020-06-24 22:04:23
4,61973223,100000004,1,2020-06-25 15:15:02


In [13]:
test.head()

,user,survey,status,date
36,60418941,100000035,0,2020-06-27 07:50:34
45,59028870,100000043,1,2020-06-27 00:41:32
80,51417258,100000074,0,2020-06-27 16:22:26
93,61203012,100000086,0,2020-06-27 11:31:07
94,56308842,100000087,0,2020-06-27 12:12:02


In [46]:
train.loc[train['status']==1, 'survey'].value_counts()

100000513    1125
100000017    1068
100000040     886
100000079     694
100000385     667
             ... 
100022408       1
100003151       1
100004528       1
100008039       1
100042425       1
Name: survey, Length: 13166, dtype: int64

In [19]:
most_popular_items = list(train.loc[train['status']==1, 'survey'].value_counts().index[0:3])

In [20]:
most_popular_items

[100000513, 100000017, 100000040]

In [30]:
actual_results = test.loc[ test['status']==1, 
                         :].groupby( ['user'] ).apply(  lambda x: list( x['survey']) ).reset_index()

actual_results.columns = ['user', 'true']

In [31]:
actual_results

,user,true
0,57141,[100002475]
1,61299,[100000427]
2,64059,"[100006888, 100010236]"
3,65613,"[100006556, 100003291, 100001173]"
4,73206,"[100016937, 100009234]"
...,...,...
25129,62072229,[100001246]
25130,62072244,[100001246]
25131,62072286,[100022773]
25132,62072334,[100005041]


In [35]:
actual_results['predicted'] = [most_popular_items]*len(actual_results)

In [36]:
actual_results

,user,true,predicted
0,57141,[100002475],"[100000513, 100000017, 100000040]"
1,61299,[100000427],"[100000513, 100000017, 100000040]"
2,64059,"[100006888, 100010236]","[100000513, 100000017, 100000040]"
3,65613,"[100006556, 100003291, 100001173]","[100000513, 100000017, 100000040]"
4,73206,"[100016937, 100009234]","[100000513, 100000017, 100000040]"
...,...,...,...
25129,62072229,[100001246],"[100000513, 100000017, 100000040]"
25130,62072244,[100001246],"[100000513, 100000017, 100000040]"
25131,62072286,[100022773],"[100000513, 100000017, 100000040]"
25132,62072334,[100005041],"[100000513, 100000017, 100000040]"


### calculate metrics

In [52]:
baseline_mapk = mapk( actual_results['true'], actual_results['predicted'] , k =3)

In [53]:
baseline_precision_at_k = np.mean(actual_results.apply( lambda x: precision_at_k(x ['true'],
                                                                                 x['predicted'], k= 3 )  , axis=1))

In [54]:
baseline_recall_at_k = np.mean(actual_results.apply( lambda x: recall_at_k(x ['true'],
                                                                                 x['predicted'], k= 3 )  , axis=1))

In [55]:
print(f'baseline map@k = {baseline_mapk:.5f} ')
print(f'baseline precision@k = {baseline_precision_at_k:.5f}')
print(f'baseline recall@k = {baseline_recall_at_k:.5f}')

baseline map@k = 0.00019 
baseline precision@k = 0.00024
baseline recall@k = 0.00040
